In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
"""By Aryan Deorah
Predicts Survival on Titanic"""

#imports pandas as pd
import pandas as pd

#imports the training and test data sets as data and test respectively
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

#prints the train columns, info regarding the train and test dataset, a description of the train data, and a few rows of data
print (train.columns)
print('_'*40)
print (train.info())
print('_'*40)
print (test.info())
print('_'*40)
print (train.describe())
print('_'*40)
print (train.head())
print('_'*40)

#prints the rate of survival for the unique inputs of the Pclass, SibSp, Sex, Parch, and Embarked
#Class, Sex, and Embarked seem to have correlation, while SibSp and Parch do not
print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print('_'*40)
print (train[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print('_'*40)
print (train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print('_'*40)
print (train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print('_'*40)
print (train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print('_'*40)

#prints the head of the data for the Ticket and Name columns, demonstrating why they should not be used
print (train.Ticket.head())
print('_'*40)
print (train.Name.head())
print('_'*40)

#drops the ticket and cabin columns due to missing inputs and no percived usefullness, passengerId is dropped from train set, creates combine, which had train and test in it
train=train.drop(['PassengerId','Ticket','Cabin'], axis=1)
test=test.drop(['Ticket','Cabin'], axis=1)
combine=[train,test]

#loops through datasets in combine and sets age values in different ranges equal to a specific number
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 5, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 5) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 35), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 35) & (dataset['Age'] <= 65), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 65, 'Age']=4

#prints the rate of survival for the unique inputs of Age, showing a sort of negative correlation
print (train[['Age', 'Survived']].groupby(['Age'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print('_'*40)

#loops through datasets in combine and creates a new column IsAlone with a 1 if the passenger was alone and 0 if they were not
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['SibSp'] + dataset['Parch'] == 0, 'IsAlone'] = 1

#prints the rate of survival for the unique inputs of IsAlone, showing a lower rate of survival for passengers traveling alone
print (train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean())
print('_'*40)

#drops Parch, SibSp, and Name for reasons mentioned above and sets combine equal to train and test again
train=train.drop(['Parch','SibSp','Name'], axis=1)
test=test.drop(['Parch','SibSp','Name'], axis=1)
combine=[train,test]

#sets freq_port equal to the port where the most passengers embarked replaces missing values with freq_port
freq_port = train.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

#replaces the port Initials with integers
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

#prints the quartile ranges for the Fare columns
print (pd.qcut(train['Fare'], 4))
print('_'*40)

#fills missing Fare values with the median fare and replaces ranges of Fare values with integers according to the quartile ranges
for dataset in combine:
    dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].dropna().median())
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

#prints the rate of survival for the unique inputs of Fare, showing a positive correlation
print (train[['Fare', 'Survived']].groupby(['Fare'], as_index=False).mean().sort_values(by='Survived', ascending=False))
print('_'*40)

#replaces male and female with integers 1 and 0
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

#prints the head of the train dataset
print (train.head())
print('_'*40)

#sets y equal to the Survived values, X equal to the train dataset with the survived value dropped, and test_X equal to test with PassengerId dropped and prints the test columns
y=train.Survived
X=train.drop("Survived", axis=1)
test_X=test.drop('PassengerId',axis=1)
print (test.columns)
#imports make_pipeline, Imputer, and SimpleImputer, cross_val_score, and 9 different models
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

#creates evaluation, an empty list, and defines the evaluate function that takes one argument model
evaluation=[]
def evaluate(model):
    
    #sets my_pipeline equal to SimpleImputer and the model, fits it to X and y, does predictions on test_X, and does cross validation scores
    my_pipeline=make_pipeline(SimpleImputer(),model)
    my_pipeline.fit(X, y)
    predictions = my_pipeline.predict(test_X)
    scores = cross_val_score(my_pipeline, X, y, scoring='neg_mean_absolute_error',cv=5)
    
    #prints the model, the cross validation scores, the mean of those scores and appends that to the evaluation list
    print ('_'*40)
    print (model)
    print(scores)
    print (scores.mean())
    evaluation.append(scores.mean())

#runs the evaluate function on all 9 models
evaluate(LogisticRegression())
evaluate(SVC())
evaluate(LinearSVC())
evaluate(RandomForestClassifier())
evaluate(KNeighborsClassifier())
evaluate(GaussianNB())
evaluate(Perceptron())
evaluate(SGDClassifier())
evaluate(DecisionTreeClassifier())

#prints the evaluation list, which shows the best cross validation score is with the SVC model
print ('_'*40)
print (evaluation)

#sets my_pipeline equal to SimpleImputer and SVC, fits it to X and y, and does predictions on test_X
my_pipeline=make_pipeline(SimpleImputer(),SVC())
my_pipeline.fit(X, y)
predictions = my_pipeline.predict(test_X)

#sets the file my_submission to a table with two columns, PassengerId and Survived, which is the predictions and makes it a csv file
my_submission = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
my_submission.to_csv('submission.csv', index=False)


['train.csv', 'gender_submission.csv', 'test.csv']
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

________________________________________
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
[-0.20111732 -0.17877095 -0.21348315 -0.23033708 -0.20903955]
-0.20654960817962492
________________________________________
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
[-0.22346369 -0.19553073 -0.15730337 -0.21910112 -0.17514124]
-0.19410803014553935
________________________________________
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
  

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If to